<a href="https://colab.research.google.com/github/partha1189/machine_learning/blob/tf_2.0_practice/CNN_food101_tfds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
train_ds, validation_ds = tfds.load('food101', split=["train[:40%]", "validation[:10%]"], as_supervised=True, batch_size=32)

In [3]:
train_ds = train_ds.map(lambda x,y : (tf.image.resize(x, (299, 299)), y))

In [4]:
validation_ds = validation_ds.map(lambda x,y : (tf.image.resize(x, (299, 299)), y))

In [5]:
train_ds

<DatasetV1Adapter shapes: ((None, 299, 299, 3), (None,)), types: (tf.float32, tf.int64)>

In [6]:
from tensorflow.keras.applications.xception import Xception

In [7]:
xception_model = Xception(include_top=True, weights='imagenet', input_shape=(299, 299, 3))

In [8]:
xception_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [9]:
layer_dict = dict([(layer.name, layer) for layer in xception_model.layers])

In [10]:
layer_dict

{'add': <tensorflow.python.keras.layers.merge.Add at 0x7fa0887e8ac8>,
 'add_1': <tensorflow.python.keras.layers.merge.Add at 0x7fa08874beb8>,
 'add_10': <tensorflow.python.keras.layers.merge.Add at 0x7fa0883e9128>,
 'add_11': <tensorflow.python.keras.layers.merge.Add at 0x7fa088350198>,
 'add_2': <tensorflow.python.keras.layers.merge.Add at 0x7fa08871cc18>,
 'add_3': <tensorflow.python.keras.layers.merge.Add at 0x7fa0886859b0>,
 'add_4': <tensorflow.python.keras.layers.merge.Add at 0x7fa088671400>,
 'add_5': <tensorflow.python.keras.layers.merge.Add at 0x7fa0885dc828>,
 'add_6': <tensorflow.python.keras.layers.merge.Add at 0x7fa088541b38>,
 'add_7': <tensorflow.python.keras.layers.merge.Add at 0x7fa08852dc88>,
 'add_8': <tensorflow.python.keras.layers.merge.Add at 0x7fa088492cf8>,
 'add_9': <tensorflow.python.keras.layers.merge.Add at 0x7fa0883fd9e8>,
 'avg_pool': <tensorflow.python.keras.layers.pooling.GlobalAveragePooling2D at 0x7fa08831a2b0>,
 'batch_normalization': <tensorflow.pyth

In [11]:
x = layer_dict['block3_pool'].output

In [12]:
x = tf.keras.layers.Conv2D(512, (3,3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(2, 2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(101, activation='softmax')(x)

In [13]:
custom_model = tf.keras.models.Model(inputs = xception_model.input, outputs = x)

In [14]:
custom_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________

In [15]:
custom_model.compile(loss='sparse_categorical_crossentropy',
                     optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
                     metrics = ['accuracy'])

In [16]:
history = custom_model.fit(train_ds, epochs=15, validation_data=validation_ds)

Epoch 1/15
      2/Unknown - 0s 227ms/step - loss: 45.0147 - accuracy: 0.0000e+00WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1050s vs `on_train_batch_end` time: 0.3484s). Check your callbacks.


947/947 [==============================] - 420s 444ms/step - loss: 4.5070 - accuracy: 0.0528 - val_loss: 3.8162 - val_accuracy: 0.1335
Epoch 2/15
947/947 [==============================] - 420s 443ms/step - loss: 3.2959 - accuracy: 0.2264 - val_loss: 3.3847 - val_accuracy: 0.2095
Epoch 3/15
947/947 [==============================] - 420s 444ms/step - loss: 1.9921 - accuracy: 0.5034 - val_loss: 3.5423 - val_accuracy: 0.2143
Epoch 4/15
947/947 [==============================] - 420s 443ms/step - loss: 0.8588 - accuracy: 0.7823 - val_loss: 4.0427 - val_accuracy: 0.2218
Epoch 5/15
947/947 [==============================] - 419s 443ms/step - loss: 0.3315 - accuracy: 0.9144 - val_loss: 4.2134 - val_accuracy: 0.2301
Epoch 6/15
947/947 [==============================] - 421s 444ms/step - loss: 0.1828 - accuracy: 0.9516 - val_loss: 4.5893 - val_accuracy: 0.2317
Epoch 7/15
947/947 [==============================] - 420s 444ms/step - loss: 0.1240 - accuracy: 0.9662 - val_loss: 4.8340 - val_accura